In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("test")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 23:16:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!wget -O data/raw/yellow_tripdata_2024-10.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [6]:
df_yellow = spark.read \
            .parquet("data/raw/yellow_tripdata_2024-10.parquet")

In [ ]:
df_yellow.show()

# Question 2 : repartition

In [ ]:
df_yellow.repartition(4).write\
    .parquet("data/pq/repartitioned")

In [10]:
df_yellow.registerTempTable("yellow")

In [ ]:
df_yellow.columns

# Question 3: count records

In [ ]:
df_count_15_oct = spark.sql("""
SELECT
    count(*)
FROM
    yellow
where tpep_pickup_datetime >= '2024-10-15 00:00:00' and
      tpep_pickup_datetime < '2024-10-16 00:00:00'                    
"""                            
)

In [ ]:
df_count_15_oct.show()

# Question 4: Longest Trip

In [ ]:
df_longest_trip = spark.sql(''' 
SELECT
    max(unix_timestamp(tpep_dropoff_datetime) - 
    unix_timestamp(tpep_pickup_datetime)) / 3600 AS longest_trip_hours 
FROM
    yellow
''')

In [ ]:
df_longest_trip.show()

In [ ]:
!wget -O data/raw/taxi_zone_lookup.csv https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [7]:
taxi_zone = spark.read\
            .option("header", "true")\
            .csv("data/raw/taxi_zone_lookup.csv")

In [8]:
taxi_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [13]:
taxi_zone.registerTempTable("taxi_zone_table")

/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [ ]:
least_frequent_pickup_zone = spark.sql('''
SELECT tz.Zone,
    COUNT(y.PULocationID) AS pickup_count
FROM
    yellow as y
INNER JOIN
    taxi_zone_table as tz
ON
    y.PULocationID = tz.LocationID
GROUP BY 
   tz.Zone
ORDER BY 
    pickup_count ASC
LIMIT 1
''')

In [22]:
least_frequent_pickup_zone.show()

+--------------------+------------+
|                Zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
+--------------------+------------+

